# <center> <span style="color:red">Music recognition </span> </center>

The purpose of this notebook is to take in music spectrograms and to recognize musical genres

## Import 

In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

## Import Model Linear and MLP 

In [2]:
import sys  
sys.path.insert(0, '../../python')

from linearModel import linearModel
from mlp import MLPModel

## Create Model 

In [3]:
lm = linearModel()
mlp = MLPModel()

## Algorithme 

In [4]:
def import_images_and_assign_labels(folder, label, X, Y):
    for file in os.listdir(folder):
        image_path = os.path.join(folder, file)
        im = Image.open(image_path)
        im = im.resize((8, 8))
        im = im.convert("RGB")
        im_arr = np.array(im)
        im_arr = np.reshape(im_arr, (8 * 8 * 3))
        X.append(im_arr)
        Y.append(label)

In [5]:
def import_dataset():
    dataset_folder = "G:/Programmes/Python/projetAnnuel/data/"
    train_folder = os.path.join(dataset_folder, "train")
    test_folder = os.path.join(dataset_folder, "test")

    X_train = []
    y_train = []
    
    import_images_and_assign_labels(
        os.path.join(train_folder, "Electro"), [1,0,0], X_train, y_train
    )
    import_images_and_assign_labels(
        os.path.join(train_folder, "Metal"), [0,1,0], X_train, y_train
    )
    
    import_images_and_assign_labels(
        os.path.join(train_folder, "Rap"), [0,0,1], X_train, y_train
    )
    
    X_test = []
    y_test = []
    import_images_and_assign_labels(
        os.path.join(test_folder, "Electro"), [1,0,0], X_test, y_test
    )
    import_images_and_assign_labels(
        os.path.join(test_folder, "Metal"), [0,1,0], X_test, y_test
    )
    
    import_images_and_assign_labels(
        os.path.join(test_folder, "Rap"), [0,0,1], X_test, y_test
    )
    
    return (np.array(X_train) / 255.0, np.array(y_train)), \
           (np.array(X_test) / 255.0, np.array(y_test))

In [6]:
def display_value(array):
    print("Sur le dataset de Train")
    cpt = 0
    number = len(array)/3
    for i in array:
        print(cpt," ",np.argmax(i)," ",i)
        cpt+=1
        if(cpt==number):
            cpt=0

In [7]:
def percentage_music_style(array):
    item1 = 0
    item2 = 0
    item3 = 0
    itemGlobal = 0 
    cpt = 0
    for i in array:
        if cpt<10:
            if np.argmax(i) == 0:
                item2 += 10
        elif cpt<20:
            if np.argmax(i) == 1: 
                item1 += 10
        else:
            if np.argmax(i) == 2:
                item3 +=10
        cpt+=1
    itemGlobal = round((item1+item2+item3)/3,2)
        
    print(f"il y a {item2}% de chanson identifié en électro")
    print(f"il y a {item1}% de chanson identifié en métal")
    print(f"il y a {item3}% de chanson identifié en rap")
    print(f"L'accuracy de notre ia est de {itemGlobal}%")

In [12]:
def run_mlp():
    (X_train, y_train), (X_test, y_test) = import_dataset()
    
    resultat1 = mlp.create_mlp_model([192,4,4,3])
    model1 = resultat1[0]
        
    flattened_dataset_inputs = []
    for p in X_train:
        for i in p:
            flattened_dataset_inputs.append(i)
            
    flattened_dataset_outputs = []
    for p in y_train:
        for i in p:
            flattened_dataset_outputs.append(i)
   
    mlp.train_classification_stochastic_gradient_backpropagation(model1,
                                                                flattened_dataset_inputs,
                                                                flattened_dataset_outputs,
                                                                alpha=1e-3,
                                                                iterations_count=200000)

    predicted_outputs = [mlp.predict_mlp_model_classification(model1, p) for p in X_test]
    

        
    #print(predicted_outputs)
    display_value(predicted_outputs)
    percentage_music_style(predicted_outputs)
    print("predicted_outputs",predicted_outputs)
#     mlp.save_mlp_classification(model1)
    
    mlp.free_MLP(resultat1)

In [13]:
run_mlp()

Sur le dataset de Train
0   2   [0.3007718324661255, 0.2677639126777649, 0.5218936204910278]
1   1   [0.4475941061973572, 0.5317878127098083, -0.015076403506100178]
2   1   [0.44273674488067627, 0.5199840068817139, 0.016710126772522926]
3   2   [0.09242604672908783, -0.07290969789028168, 0.8096753358840942]
4   2   [0.3121228814125061, 0.28903356194496155, 0.48498955368995667]
5   2   [0.12888234853744507, -0.017648987472057343, 0.7851033806800842]
6   2   [0.2346423715353012, 0.15503303706645966, 0.6553587913513184]
7   1   [0.4518733620643616, 0.5415374636650085, -0.03815017268061638]
8   1   [0.42962419986724854, 0.5029703378677368, 0.04014020785689354]
9   1   [0.44799622893333435, 0.5347030162811279, -0.024907011538743973]
0   1   [0.45650771260261536, 0.5539143085479736, -0.06528174132108688]
1   1   [0.4520954191684723, 0.5433770418167114, -0.04378075525164604]
2   1   [0.44453760981559753, 0.5278515815734863, -0.009785131551325321]
3   1   [0.4507449269294739, 0.542394816875457

In [10]:
def load_mlp():
    (X_train, y_train), (X_test, y_test) = import_dataset()
    
    resultat2 = mlp.load_mlp_model("../../Save/mlp_192_4_4_4_3_classification_2021717234718.json")
    model2 = resultat2[0]
        
    predicted_outputs = [mlp.predict_mlp_model_classification(model2, p) for p in X_test]

    display_value(predicted_outputs)
    percentage_music_style(predicted_outputs)
    
    mlp.free_MLP(resultat2)

In [11]:
load_mlp()

Sur le dataset de Train
0   0   [0.8075122833251953, 0.048556555062532425, 0.05690764635801315]
1   0   [0.7278856039047241, 0.08244484663009644, 0.15438894927501678]
2   0   [0.6801071166992188, 0.42613130807876587, 0.01486883219331503]
3   2   [0.07987909018993378, 0.08412139117717743, 0.9121149182319641]
4   0   [0.6726241111755371, 0.08840977400541306, 0.224213644862175]
5   2   [0.0796002522110939, 0.08535412698984146, 0.912695050239563]
6   2   [0.28025054931640625, 0.023924939334392548, 0.6956740021705627]
7   0   [0.8180463314056396, 0.05788940191268921, 0.03643370792269707]
8   0   [0.7101330161094666, 0.36984747648239136, 0.017267199233174324]
9   0   [0.8019040822982788, 0.05261043831706047, 0.06509718298912048]
0   1   [0.2124222069978714, 0.8279378414154053, -0.0017302242340520024]
1   1   [0.3571777045726776, 0.7574736475944519, 0.0011300737969577312]
2   1   [0.026502005755901337, 0.9066774845123291, 0.006271928083151579]
3   1   [0.024324268102645874, 0.9055659174919128